In [1]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akshara/anacon

In [2]:
from keras.layers import Conv1D,Dense
from keras.models import Sequential


In [3]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
test_val = np.load('test_val.npy')
y_test = np.load('y_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [4]:
y_train.shape

(37065, 20)

In [5]:

len(x_train[3])

1011

In [6]:
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))

In [7]:
from keras.engine import InputSpec, Layer

class squash_function(Layer):
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector

In [8]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K
class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPooling1D, Dropout, Conv1D,GlobalAveragePooling1D,Bidirectional,GRU,LSTM,concatenate,Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention

filter_length = 128
num_classes = 20
max_words = 111396
maxlen = 1011

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

input1 = Input(shape=(maxlen,))
print(input1.shape)
 
x = Embedding(max_words, 20, input_length=maxlen)(input1)

x = squash_function()(x)

x = Conv1D(96, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)

x = squash_function()(x)
print(x.shape)

attn1 = AttentionWeightedAverage()(x)

conv1 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv2 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv3 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat = concatenate([conv2,conv1])
cat2 = concatenate([cat,conv3])


conv_1 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_2 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_3 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat_1 = concatenate([conv_2,conv_1])
cat_2 = concatenate([cat_1,conv_3])

conv4 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv5 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv6 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat4 = concatenate([conv4,conv5])
cat5 = concatenate([cat,conv6])


conv_4 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_5 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_6 = Conv1D(32, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat_4 = concatenate([conv_4,conv_5])
cat_5 = concatenate([cat_4,conv_6])

m = concatenate([cat2,cat_2])
n = concatenate([cat_5,cat5])

x = concatenate([m,n])

x = squash_function()(x)

maxpool = GlobalMaxPooling1D()(x)
attn = AttentionWeightedAverage()(x)
avg = GlobalAveragePooling1D()(x)

l = concatenate([maxpool,avg])
s = concatenate([l,attn])
x = concatenate([s,attn1])
x = squash_function()(x)
 
preds = Dense(num_classes, activation="sigmoid")(x)
 
model = keras.Model(input1, preds)
 
model.summary()
 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Hamming_loss])


# callbacks = [
#     ReduceLROnPlateau(), 
#     EarlyStopping(patience=4)
# ]

# history = model.fit(x_train, y_train,
#                     epochs=10,
#                     batch_size=32,
#                     validation_split=0.1,
#                     callbacks=callbacks)



(?, 1011)

(?, 1009, 96)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1011)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1011, 20)     2227920     input_1[0][0]                    
__________________________________________________________________________________________________
squash_function_1 (squash_funct (None, 1011, 20)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1009, 96)     5856        squash_function_1[0][0]          
__________________________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss

def _train_model(model, batch_size, train_x, train_y, test_val, y_val):
    num_labels = train_y.shape[1]
    patience = 7
    best_loss = -1
    best_weights = None
    best_epoch = 0
    
    current_epoch = 0
    
    while True:
        model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
        y_pred = model.predict(test_val, batch_size=batch_size)

        total_loss = 0
        for j in range(num_labels):
            loss = log_loss(y_val[:, j], y_pred[:, j])
            total_loss += loss

        total_loss /= num_labels

        print("Epoch {0} loss {1} best_loss {2}".format(current_epoch, total_loss, best_loss))

        current_epoch += 1
        if total_loss < best_loss or best_loss == -1:
            best_loss = total_loss
            best_weights = model.get_weights()
            best_epoch = current_epoch
        else:
            if current_epoch - best_epoch == patience:
                break

    model.set_weights(best_weights)
    return model

In [11]:
batch_size = 32
history = _train_model(model,batch_size,x_train,y_train,test_val,y_val)


Epoch 1/1
37065/37065 [==============================] - 39s 1ms/step - loss: 0.4158 - Hamming_loss: 0.2024
Epoch 0 loss 0.3864687438918085 best_loss -1
Epoch 1/1
37065/37065 [==============================] - 36s 964us/step - loss: 0.3685 - Hamming_loss: 0.1725
Epoch 1 loss 0.3704671500490633 best_loss 0.3864687438918085
Epoch 1/1
37065/37065 [==============================] - 36s 966us/step - loss: 0.3482 - Hamming_loss: 0.1598
Epoch 2 loss 0.36674736978729033 best_loss 0.3704671500490633
Epoch 1/1
37065/37065 [==============================] - 36s 967us/step - loss: 0.3272 - Hamming_loss: 0.1485
Epoch 3 loss 0.3731110301731607 best_loss 0.36674736978729033
Epoch 1/1
37065/37065 [==============================] - 36s 966us/step - loss: 0.3009 - Hamming_loss: 0.1343
Epoch 4 loss 0.38715635909287216 best_loss 0.36674736978729033
Epoch 1/1
37065/37065 [==============================] - 36s 966us/step - loss: 0.2690 - Hamming_loss: 0.1177
Epoch 5 loss nan best_loss 0.36674736978729033
E

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

37065/37065 [==============================] - 36s 964us/step - loss: 0.2345 - Hamming_loss: 0.1001
Epoch 6 loss nan best_loss 0.36674736978729033
Epoch 1/1
   96/37065 [..............................] - ETA: 36s - loss: 0.1978 - Hamming_loss: 0.0766

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

37065/37065 [==============================] - 36s 969us/step - loss: 0.2003 - Hamming_loss: 0.0831
Epoch 7 loss nan best_loss 0.36674736978729033
Epoch 1/1
   96/37065 [..............................] - ETA: 35s - loss: 0.1454 - Hamming_loss: 0.0583

...

37065/37065 [==============================] - 36s 965us/step - loss: 0.1686 - Hamming_loss: 0.0682
Epoch 8 loss nan best_loss 0.36674736978729033
Epoch 1/1
   96/37065 [..............................] - ETA: 36s - loss: 0.1489 - Hamming_loss: 0.0604

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

37065/37065 [==============================] - 36s 964us/step - loss: 0.1413 - Hamming_loss: 0.0557


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 9 loss nan best_loss 0.36674736978729033


In [12]:
y_pred = history.predict(x_test, batch_size=batch_size)

In [13]:
conv1.shape

TensorShape([Dimension(None), Dimension(1008), Dimension(32)])

In [14]:
conv2.shape

TensorShape([Dimension(None), Dimension(1008), Dimension(32)])

In [15]:
y_val

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [16]:
y_test.shape

(8177, 20)

In [17]:
print(y_pred[35],y_val[35])

[0.00440019 0.00816673 0.01220965 0.00623998 0.00309649 0.01961866
 0.04502234 0.01499572 0.02243465 0.00207087 0.0016745  0.02980751
 0.0063543  0.20442188 0.6924015  0.06060821 0.00482515 0.00569949
 0.00610349 0.98821497] [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0]


In [18]:
(7.8654468e-01+           
5.0963247e-01+
9.6992075e-01+
5.8361495e-01+
3.5057139e-01+
5.7766867e-01)/6

0.6296588183333333

In [19]:
y_pred_round = np.where(y_pred>0.55,1,0)#0.6 enough na lets see....happy ya

###### y_train.shape

In [20]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_test,y_pred_round,average='micro'))
print(jaccard_score(y_test,y_pred_round,average='macro'))
print(roc_auc_score(y_test,y_pred_round))
print(hamming_loss(y_test,y_pred_round))

0.4911807580174927
0.37797459905981046
0.6696873909013044
0.17074721780604132


In [21]:
from keras.models import load_model
model.save('conv5.h5')